In [1]:
import os 
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
import route_dynamics.route_elevation.base_df as base
import route_dynamics.route_energy.longi_dynam_model as ldm
import route_dynamics.route_energy.accel as accel
from route_dynamics.route_riders import route_riders as ride
from route_dynamics.route_visualizer import visualizer as vis
import route_dynamics.route_energy.knn as knn

In [4]:
#User defines what routes they want to evaluate
rt_list = [22]

#Import route shapefile
shapefile_name = '../data/rt' + str(rt_list[0]) + '_pts.shp'

#Import bus stops shapefile
stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

trip183 = pd.read_csv("../data/Trip183.csv", usecols = ['SignRt', 'InOut', 'KeyTrip', 'BusType', 'Seats', 
                     'Period', 'AnnRides']) # KCM Data
trip183unsum = pd.read_csv("../data/Zon183Unsum.csv", usecols = ['Route', 'Dir', 'Trip_ID', 'InOut', 'STOP_SEQ', 'STOP_ID',
                     'Period', 'AveOn', 'AveOff', 'AveLd', 'Obs'])

#Acceleration profile
a_prof =pd.read_csv('../data/acceleration.csv', names=['time (s)', 'accel. (g)'])

In [5]:
a_prof.head()

,time (s),accel. (g)
0,0.000000,0.000556
1,0.917431,0.115556
2,1.972477,0.123889
3,2.935780,0.126111
4,4.036697,0.125556


In [8]:
route_df = base.wrapper(shapefile_name, 6, 6)
route_df.head()

,geometry,distance,elevation,grade
0,POINT (-122.38813 47.56295),0.0000,99.398090,0.000000
1,POINT (-122.38798 47.56295),10.9728,99.874774,0.043442
2,POINT (-122.38784 47.56295),21.9456,100.272354,0.036233
3,POINT (-122.38769 47.56295),32.9184,100.594760,0.029382
4,POINT (-122.38754 47.56295),43.8912,100.845922,0.022889


In [ ]:
route_df.head()

In [ ]:
df_22, riders_22, mass_22 = ride.route_ridership('PM', 22)

In [ ]:
stop_coord, rider_coord = ride.stop_coord(22, riders_22)

In [ ]:
stop_coords = stop_coord.coordinates.values

In [ ]:
stop_nn_indicies, stop_coord_nn = knn.find_knn(
1,
route_df.geometry.values,
stop_coords
)
# the 'jth' element of stop_nn_indicies also selects the

route_df = route_df.assign(
is_bus_stop = ([False] * len(route_df.index))
)

for i in stop_nn_indicies.ravel():
    route_df.at[i, 'is_bus_stop'] = True

In [ ]:
route_df.is_bus_stop.value_counts()

In [ ]:
a, v, x_ls, x_ns, t  = accel.accel_dynamics(route_df, a_prof, -0.4, 15)

In [ ]:
route_df['a'] = a

In [ ]:
route_df['a'].value_counts()

In [ ]:
a_prof